# Problem definition

We wish to minimize
$$ I(u,v) = \frac{\theta}{2} \int_{\omega} |\nabla_s u + \tfrac{1}{2} \nabla v \otimes \nabla v|^{2} \mathrm{d}x
   + \frac{1}{24} \int_{\omega} |\nabla^2 v - \mathrm{Id}|^{2} \mathrm{d}x. $$

Because we only have $C^0$ elements we minimize instead

$$ J(u,v) = \frac{\theta}{2} \int_{\omega} |\nabla_s u + \tfrac{1}{2} v \otimes v|^{2} \mathrm{d}x 
          + \frac{1}{24} \int_{\omega} |\nabla v - \mathrm{Id}|^{2} \mathrm{d}x 
          + \frac{\mu}{2} \int_{\omega} |\mathrm{curl}\ v|^{2} \mathrm{d}x, $$

then recover the vertical displacements (up to a constant) by minimizing

$$ F(p,q) = \tfrac{1}{2} || \nabla p - q ||^2 + \tfrac{1}{2} || q - v ||^2. $$

This we do by solving the linear problem $D F = 0$.

Minimization of the energy is done via gradient descent and a line search.

In [ ]:
from dolfin import *
import mshr
import numpy as np
import matplotlib.pyplot as pl
%matplotlib inline

parameters["form_compiler"]["optimize"]     = True
parameters["form_compiler"]["cpp_optimize"] = True

In [ ]:
def compute_potential(v: Function, U: FunctionSpace) -> Function:
    """ Takes a gradient and computes its potential (up to a constant)
    Note that we would need to set Dirichlet conditions on the
    potential to fix the constant.

    Arguments
    ---------
        v: gradient
        U: space for the potential
    """
    msh = v.function_space().mesh()
    PE = U.ufl_element()
    QE = v.function_space().ufl_element()
    W = FunctionSpace(msh, PE*QE)

    class GradientsBoundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary
    class ValuesBoundary(SubDomain):
        def inside(self, x, on_boundary):
            return False

    bcP = DirichletBC(W.sub(0), Constant(0.0), ValuesBoundary())  # void...
    bcQ = DirichletBC(W.sub(1), v, GradientsBoundary())
    p, q = TrialFunctions(W)
    phi, psi = TestFunctions(W)
    a = inner(grad(p) - q, grad(phi))*dx - inner(grad(p) - q, psi)*dx + inner(q, psi)*dx
    L = inner(v, psi)*dx
    w = Function(W)
    solve(a == L, w, [bcP, bcQ])

    ret = Function(U)
    fa = FunctionAssigner(U, W.sub(0))
    fa.assign(ret, w.sub(0))
    #ret = w.sub(0, deepcopy=True)
    ret.rename("pot", "potential")    
    return ret

def test_potential(eps=1e-6):
    print("Testing potential (with integration constant hack)... ", end='')
    msh = UnitSquareMesh(10, 10)
    U = FunctionSpace(msh, "Lagrange", 3)
    u = interpolate(Expression(("x[0]*x[0] + x[1]*x[1]"), element=U.ufl_element()), U)
    V = VectorFunctionSpace(msh, "Lagrange", 2, dim=2)    
    v = interpolate(Expression(("2*x[0]", "2*x[1]"), element=V.ufl_element()), V)
    p = compute_potential(v, U)
    
    # HACK: fix the integration constant 
    hack = norm(project(u - p, U))
    p = project(p + Constant(hack), U)
    test1 = project(u - p, U)
    test2 = project(v - grad(p), V)
    print("OK." if norm(test1) < eps and norm(test2) < eps else "FAILED.")

In [ ]:
domain = mshr.Circle(Point(0.0,0.0), 1, 20)
msh = mshr.generate_mesh(domain, 20)

In [ ]:
class InitialDisplacements(Expression):     
    def eval(self, values, x):
        values[0] = 0.0
        values[1] = 0.0
        values[2] = 0.3*x[0]
        values[3] = x[1]
    def value_shape(self):
        return (4,)

class DirichletBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return False
        #return near(x[0], 0.0) and near(x[1], 0.0)
        #return near(x[0], 0.0) and -0.5 <= x[1] and x[1] <= -0.5+msh.hmin()*1.1 and on_boundary
        #return near(x[0], 0.0) or near(x[1], -0.5) and on_boundary

We store outputs from different runs in a global array

In [ ]:
if locals().get('history') is None:
    history = []

In plane displacements and gradient of out of plane displacements form a mixed function space. We also have another scalar space where to project the potential for the out of plane displacements

In [ ]:
UE = VectorElement("Lagrange", msh.ufl_cell(), 2, dim=2)        # in plane displacements (IPD)
VE = VectorElement("Lagrange", msh.ufl_cell(), 2, dim=2)        # Gradients of out of plane displacements (OPD)
W = FunctionSpace(msh, UE*VE)
V = FunctionSpace(msh, "Lagrange", 2)                           # will store out of plane displacements

bcU = DirichletBC(W.sub(0), Constant((0.0, 0.0)), DirichletBoundary())
bcV = DirichletBC(W.sub(1), Constant((0.0, 0.0)), DirichletBoundary())

We gather in-plane and out-of-plane displacements into one function for visualization with ParaView.

In [ ]:
P = VectorFunctionSpace(msh, "Lagrange", 2, dim=3)
fax = FunctionAssigner(P.sub(0), W.sub(0).sub(0))
fay = FunctionAssigner(P.sub(1), W.sub(0).sub(1))
faz = FunctionAssigner(P.sub(2), V)

disp = Function(P)
disp.rename("disp", "displacement")

In [ ]:
file = File("descent-curl.pvd")

w = Function(W)
w_ = Function(W)
u, v  = w.split()
u_, v_ = w_.split()

w_init = InitialDisplacements(degree=1)
w.interpolate(w_init)
w_.interpolate(w_init)

def eps(u):
    return (grad(u) + grad(u).T)/2.0

e_stop = msh.hmin()*1e-6
max_steps = 400
max_line_search_steps = 20
step = 0
tau = 1        # Maximal step size
omega = 0.25   # Gradient descent fudge factor in (0, 1/2)
mu = 1e4
theta = 1.0e2
_hist = {'mu': mu, 'theta': theta, 'e_stop' : e_stop,
         'J':[], 'alpha':[], 'du':[], 'dv':[]}

Id = Identity(2)
zero_energy = assemble((1./24)*inner(Id, Id)*dx(msh))
def energy(u, v, mu=mu):
    J = (theta/2)*inner(eps(u)+outer(v, v)/2, eps(u)+outer(v, v)/2)*dx(msh) \
        + (1./24)*inner(grad(v) - Id, grad(v) - Id)*dx(msh) \
        + mu*inner(curl(v), curl(v))*dx(msh)
    return assemble(J)

phi, psi = TestFunctions(W)
# CAREFUL!! Picking the right scalar product here is essential
# Recall the issues with boundary values: integrate partially and only boundary terms survive...
#dtu, dtv = TrialFunctions(W)
#L = inner(grad(dtu), grad(phi))*dx(msh) + inner(grad(dtv), grad(psi))*dx(msh)
dtw = TrialFunction(W)
z = TestFunction(W)
L = inner(dtw, z)*dx+inner(grad(dtw), grad(z))*dx

dw = Function(W)
du, dv = dw.split()

# Output initial condition
opd = compute_potential(v, V)
fax.assign(disp.sub(0), u.sub(0))
fay.assign(disp.sub(1), u.sub(1))
faz.assign(disp.sub(2), opd)
file << (disp, float(step))

cur_energy = energy(u, v)
alpha = tau
ndu = ndv = 1.0

print("Solving with theta = %.3e, mu = %.3e, for at most %d steps." % (theta, mu, max_steps))
while alpha*(ndu**2+ndv**2) > e_stop and step < max_steps:
    print("Step %d, energy = %.3e, curl = %.3e" % (step, cur_energy, assemble(curl(v_)*dx)))
    
    dJ = theta*inner(eps(u_)+outer(v_, v_)/2, eps(phi))*dx(msh) \
        + theta*inner(eps(u_)+outer(v_, v_)/2, outer(v_, psi))*dx(msh) \
        + (1./12)*inner(grad(v_)-Id, grad(psi))*dx(msh) \
        + 2*mu*inner(curl(v_), curl(psi))*dx(msh)

    print("\tSolving...", end='')
    solve(L == -dJ, dw, [bcU, bcV])
    
    # dw is never reassigned to a new object so it's ok to reuse du, dv without splitting
    ndu = norm(du)
    ndv = norm(dv)
    
    print(" done with |du| = %.3f, |dv| = %.3f" % (ndu, ndv))

    # line search
    new_energy = 0
    #alpha = tau
    print("\tSearching... ", end='')
    while True:
        w = project(w_ + alpha*dw, W)
        u, v = w.split()
        new_energy = energy(u, v)
        if new_energy <= cur_energy - omega*alpha*(ndu**2+ndv**2):
            print(" alpha = %e" % alpha)
            _hist['J'].append(cur_energy)
            _hist['alpha'].append(alpha)
            _hist['du'].append(ndu)
            _hist['dv'].append(ndv)
            cur_energy = new_energy
            alpha /= 0.5  # Use a larger alpha for the next main iteration
            break
        if alpha < (1./2)**max_line_search_steps:
            raise Exception("Line search failed after %d steps" % max_line_search_steps)
        alpha *= 0.5  # Repeat with smaller alpha

    step += 1

    opd = compute_potential(v, V)
    fax.assign(disp.sub(0), u.sub(0))
    fay.assign(disp.sub(1), u.sub(1))
    faz.assign(disp.sub(2), opd)
    file << (disp, float(step))

    w_.vector()[:] = w.vector()
    u_, v_ = w_.split()

print("Done after %d steps" % step)

In [ ]:
_hist['steps'] = step
_hist['u'] = u
_hist['v'] = v
_hist['dtu'] = du
_hist['dtv'] = dv
history.append(_hist)

## History

In [ ]:
pl.figure(figsize=(18,8))
pl.subplot(2,2,1)
pl.plot(history[-1]['du'])
pl.title('$d_{t}u$')
pl.subplot(2,2,2)
pl.plot(history[-1]['dv'])
pl.title('$d_{t}v$')
pl.subplot(2,2,3)
pl.plot(np.log(history[-1]['alpha'][1:]))
pl.title('$log\ \\alpha_t$')
pl.subplot(2,2,4)
pl.plot(history[-1]['J'])
pl.title("Energy")
print("Last run: theta = %.2e, mu = %.2e" % (history[-1]['theta'], history[-1]['mu']))

## Solution and last update

In [ ]:
pl.figure(figsize=(18,18))
pl.subplot(2,2,1)
plot(history[-1]['u'], title="$u_{\\theta}$ at last timestep, norm = %.2e" % norm(history[-1]['u']))
pl.subplot(2,2,2)
plot(history[-1]['v'], title="$v_{\\theta}$ at last timestep, norm = %.2e" % norm(history[-1]['v']))
pl.subplot(2,2,3)
plot(history[-1]['dtu'], title="$du_{\\theta}$ at last timestep, norm = %.2e" % norm(history[-1]['dtu']))
pl.subplot(2,2,4)
_ = plot(history[-1]['dtv'], title="$dv_{\\theta}$ at last timestep, norm = %.2e" % norm(history[-1]['dtv']))

# Mixed out-of-plane displacements (FIXME)

In [ ]:
class InitialDisplacements(Expression):     
    def eval(self, values, x):
        values[0] = 0.0
        values[1] = -x[1]/10.0
        values[2] = 0.5*x[0]**2*(1-x[0])**2*sin(4*DOLFIN_PI*x[1])
        values[3] = x[0]*(1-x[0])*(1-2*x[0])*sin(4*DOLFIN_PI*x[1])
        values[4] = 2*DOLFIN_PI*x[0]**2*(1-x[0])**2*cos(4*DOLFIN_PI*x[1])
    def value_shape(self):
        return (5,)

msh = RectangleMesh(Point(0.0, -0.5), Point(1.0, 0.5), 20, 20)

UE = VectorElement("Lagrange", msh.ufl_cell(), 1, dim=2)        # in plane displacements (IPD)
VE = FiniteElement("Lagrange", msh.ufl_cell(), 1)
VVE = VectorElement("Lagrange", msh.ufl_cell(), 1, dim=2)        # Gradients of out of plane displacements (OPD)
ME = MixedElement([UE, VE, VVE])
W = FunctionSpace(msh, ME)

bc = DirichletBC(W, Expression(("0.0", "-x[1]/10", "0.0", "0.0", "0.0"), element=ME),
                 DirichletBoundary())
w = Function(W)
w_ = Function(W)
u, d, v  = w.split()
u_, d_, v_ = w_.split()

w_init = InitialDisplacements(degree=1)
w.interpolate(w_init)
w_.interpolate(w_init)

def eps(u):
    return (grad(u) + grad(u).T)/2.0

e_stop = msh.hmin()*1e-4

file = File("steepest-descent-grad.pvd")

mu = Constant(1e4)
Id = Identity(2)

zero_energy = assemble((1./24)*inner(Id, Id)*dx(msh))
def energy(u, d, v, mu=mu):
    J = (1./2)*inner(eps(u)+outer(v, v)/2, eps(u)+outer(v, v)/2)*dx \
        + (1./24)*inner(grad(v) - Id, grad(v) - Id)*dx \
        + (1./2)*mu*inner(v, v)*dx \
        + (1./2)*mu*inner(v - grad(d), v - grad(d))*dx
        
    return assemble(J)

phi, eta, psi = TestFunctions(W)

dtz = TrialFunction(W)
z = TestFunction(W)
L = inner(dtz, z)*dx

dw = Function(W)
du, dd, dv = dw.split()
step = 0
max_line_steps = 30
tau = 1
omega = 0.25  # fudge factor

cur_energy = energy(u, d, v)

alpha = ndu = ndd = ndv = 1.0
history = {'J':[], 'alpha':[]}

while alpha*(ndu**2+ndd**2+ndv**2) > e_stop and step < 100:
    print("Step %d, energy = %.3e, curl = %.3e" % (step, cur_energy, assemble(curl(w.sub(2))*dx)))
    dJ = inner(eps(u_) + outer(v_, v_)/2, eps(phi))*dx \
        + inner(eps(u_) + outer(v_, v_)/2, outer(v_, psi))*dx \
        + (1./12)*inner(grad(v_)-Id, grad(psi))*dx \
        + mu*inner(v_, psi)*dx \
        + mu*inner(v_ - grad(d_), phi)*dx + mu*inner(v_ - grad(d_), grad(eta))*dx

    print("\tSolving...", end='')
    solve(L == -dJ, dw, bc)
    
    # dw is not assigned to a new object so it's ok to reuse du, dd, dv without splitting
    ndu = norm(du)
    ndd = norm(dd)
    ndv = norm(dv)
    
    print(" done with |du| = %.3f, |dd| = %.3f, |dv| = %.3f" % (ndu, ndd, ndv), end='')

    # line search
    new_energy = 0
    alpha = tau
    while True:
        w = project(w_ + alpha*dw, W)
        u, d, v = w.split()
        new_energy = energy(u, d, v)
        #print("New energy: %e, alpha = %.2e" % (new_energy, alpha))
        if new_energy <= cur_energy - omega*alpha*(ndu**2+ndd**2+ndv**2):
            print(", alpha = %e" % alpha)
            history['J'].append(cur_energy)
            history['alpha'].append(alpha)
            cur_energy = new_energy
            break
        if alpha < (1./2)**max_line_steps:
            raise Exception("Line search failed after %d steps" % max_line_steps)
        alpha *= 0.5
    w.rename("disp", "displacement")
    file << (w.split()[0], float(step))
    file << (w.split()[1], float(step))
    
    w_.vector()[:] = w.vector()
    u_, d_, v_ = w_.split()

    step += 1